In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def k_means(X, k, max_iterations=100, tol=1e-4):
    centroids = X[np.random.choice(X.shape[0], k, replace=False)]

    for iteration in range(max_iterations):
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)

        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])

        if np.linalg.norm(new_centroids - centroids) < tol:
            break

        centroids = new_centroids

    return centroids, labels

def read_tsp_file(file_path):
    tsp_data = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()

    coordinates = []
    for line in lines:
        if line.startswith("NODE_COORD_SECTION"):
            for i in range(lines.index(line) + 1, len(lines)):
                if lines[i].strip() == "EOF":
                    break
                parts = lines[i].split()
                x = float(parts[1])
                y = float(parts[2])
                coordinates.append((x, y))
            tsp_data['coordinates'] = coordinates

    return tsp_data

def create_data_model(cluster_points):
    data = {}
    data['distance_matrix'] = create_distance_matrix(cluster_points)
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

def create_distance_matrix(points):
    size = len(points)
    distance_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if i != j:
                distance_matrix[i][j] = np.linalg.norm(points[i] - points[j])
    return distance_matrix

def solve_tsp(data, time_limit=1):
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.time_limit.seconds = time_limit
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        return get_solution(manager, routing, solution)
    else:
        return None

def get_solution(manager, routing, solution):
    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))
    return route

def calculate_route_distance(route, distance_matrix):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i]][route[i+1]]
    total_distance += distance_matrix[route[-1]][route[0]]
    return total_distance

def merge_routes(route1, route2, distance_matrix):
    min_distance = float('inf')
    best_i, best_j = -1, -1

    print(route1)
    print(route2)
    for i in range(len(route1) - 1):
        for j in range(len(route2) - 1):
            dist1 = distance_matrix[route1[i]][route2[j]] + distance_matrix[route2[j+1]][route1[i+1]] - distance_matrix[route1[i]][route1[i+1]] - distance_matrix[route2[j]][route2[j+1]]
            dist2 = distance_matrix[route1[i]][route2[j+1]] + distance_matrix[route2[j]][route1[i+1]] - distance_matrix[route1[i]][route1[i+1]] - distance_matrix[route2[j]][route2[j+1]]
            if min(dist1, dist2) < min_distance:
                min_distance = min(dist1, dist2)
                best_i, best_j = i, j
    
    print(min_distance)
    
    if dist1 < dist2:
        print('ij')
        new_route = route1[:best_i+1] + route2[best_j::-1] + route2[-1:best_j:-1] + route1[best_i+1:]
    else:
        print('ij+1')
        new_route = route1[:best_i+1] + route2[best_j+1:] + route2[:best_j+1] + route1[best_i+1:]

    new_route = list(dict.fromkeys(new_route)) 
    return new_route

def plot_all_routes(coordinates, routes, total_distance, title):
    plt.figure(figsize=(12, 8))
    plt.scatter(coordinates[:, 0], coordinates[:, 1], c='black', zorder=1)
    
    colors = plt.cm.get_cmap('tab10', len(routes))
    for i, route in enumerate(routes):
        color = colors(i)
        for j in range(len(route) - 1):
            start, end = route[j], route[j+1]
            plt.plot([coordinates[start][0], coordinates[end][0]], 
                     [coordinates[start][1], coordinates[end][1]], 
                     color=color, linewidth=2, zorder=2)
        plt.plot([coordinates[route[-1]][0], coordinates[route[0]][0]], 
                 [coordinates[route[-1]][1], coordinates[route[0]][1]], 
                 color=color, linewidth=2, zorder=2, linestyle='--')

    plt.title(f"{title}\nTotal Route Distance: {total_distance:.2f}")
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True)
    plt.show()

    
file_path = 'pr144.tsp'
tsp_data = read_tsp_file(file_path)
coordinates = np.array(tsp_data['coordinates'])

num_clusters = 6

centroids, labels = k_means(coordinates, num_clusters)

for i in range(num_clusters):
    cluster_indices = [index for index, label in enumerate(labels) if label == i]
    print(f'Cluster {i + 1} contains vertices with indices: {cluster_indices}')

cluster_routes = []
cluster_centroids = []
total_distance = 0

for i in range(num_clusters):
    cluster_points = coordinates[labels == i]

    data = create_data_model(cluster_points)
    optimal_route = solve_tsp(data)

    if optimal_route:
        cluster_routes.append([np.where((coordinates == cluster_points[label]).all(axis=1))[0][0] for label in optimal_route])
        cluster_centroids.append(centroids[i])

        route_distance = calculate_route_distance(optimal_route, data['distance_matrix'])
        total_distance += route_distance
        print(f'Distance of the route for Cluster {i + 1}: {route_distance:.2f}')
        
        cluster_indices = [index for index, label in enumerate(labels) if label == i]
        cluster_route = [cluster_indices[node] for node in optimal_route]
        print(f'Optimal route for Cluster {i + 1}: {cluster_route}')
    else:
        cluster_routes.append([])
        cluster_centroids.append(centroids[i])

plot_all_routes(coordinates, cluster_routes, total_distance, 'Initial Routes for All Clusters')

cluster_routes_copy1 = cluster_routes.copy()
cluster_centroids_copy1 = cluster_centroids.copy()

cluster_routes_copy2 = cluster_routes.copy()
cluster_centroids_copy2 = cluster_centroids.copy()

distance_matrix = create_distance_matrix(coordinates)
merge_step = 1

while len(cluster_routes_copy1) > 1:
    min_distance = float('inf')
    best_pair = None

    for i in range(len(cluster_centroids_copy1) - 1):
        for j in range(i + 1, len(cluster_centroids_copy1)):
            dist = np.linalg.norm(cluster_centroids_copy1[i] - cluster_centroids_copy1[j])
            if dist < min_distance:
                min_distance = dist
                best_pair = (i, j)

    if best_pair:
        i, j = best_pair
        route1 = cluster_routes_copy1.pop(i)
        route2 = cluster_routes_copy1.pop(j-1 if j > i else j)
        merged_route = merge_routes(route1, route2, distance_matrix)
        cluster_routes_copy1.append(merged_route)
        
        new_centroid = np.mean([coordinates[point] for point in merged_route], axis=0)
        cluster_centroids_copy1.pop(i)
        cluster_centroids_copy1.pop(j-1 if j > i else j)
        cluster_centroids_copy1.append(new_centroid)
        
        total_distance_copy1 = sum(calculate_route_distance(route, distance_matrix) for route in cluster_routes_copy1)
        
        plot_all_routes(coordinates, cluster_routes_copy1, total_distance_copy1, f'Merged Routes After Step {merge_step} (Strategy 1)')
        merge_step += 1

final_route_copy1 = cluster_routes_copy1[0]

print(f'Final merged route (Strategy 1): {final_route_copy1}')
print(f'Number of points in final route (Strategy 1): {len(final_route_copy1)}')
plot_all_routes(coordinates, [final_route_copy1], total_distance_copy1, 'Final Merged Route (Strategy 1)')

merge_step = 1
while len(cluster_routes_copy2) > 1:
    min_distance = float('inf')
    best_pair = None

    for i in range(len(cluster_centroids_copy2) - 1):
        for j in range(i + 1, len(cluster_centroids_copy2)):
            dist = np.linalg.norm(cluster_centroids_copy2[i] - cluster_centroids_copy2[j])
            if dist < min_distance:
                min_distance = dist
                best_pair = (i, j)

    if best_pair:
        i, j = best_pair
        route1 = cluster_routes_copy2.pop(i)
        route2 = cluster_routes_copy2.pop(j-1 if j > i else j)
        merged_route = merge_routes(route1, route2, distance_matrix)
        cluster_routes_copy2.append(merged_route)
        
        new_centroid = np.mean([coordinates[point] for point in merged_route], axis=0)
        cluster_centroids_copy2.pop(i)
        cluster_centroids_copy2.pop(j-1 if j > i else j)
        cluster_centroids_copy2.append(new_centroid)
        
        total_distance_copy2 = sum(calculate_route_distance(route, distance_matrix) for route in cluster_routes_copy2)
        
        plot_all_routes(coordinates, cluster_routes_copy2, total_distance_copy2, f'Merged Routes After Step {merge_step} (Strategy 2)')
        merge_step += 1

        while len(cluster_routes_copy2) > 1:
            min_distance = float('inf')
            closest_cluster_index = -1
            for i, centroid in enumerate(cluster_centroids_copy2[:-1]):
                dist = np.linalg.norm(centroid - new_centroid)
                if dist < min_distance:
                    min_distance = dist
                    closest_cluster_index = i
            
            if closest_cluster_index != -1:
                route1 = cluster_routes_copy2.pop(-1)
                route2 = cluster_routes_copy2.pop(closest_cluster_index)
                merged_route = merge_routes(route1, route2, distance_matrix)
                cluster_routes_copy2.append(merged_route)

                new_centroid = np.mean([coordinates[point] for point in merged_route], axis=0)
                cluster_centroids_copy2.pop(-1)
                cluster_centroids_copy2.pop(closest_cluster_index)
                cluster_centroids_copy2.append(new_centroid)
                
                total_distance_copy2 = sum(calculate_route_distance(route, distance_matrix) for route in cluster_routes_copy2)
                plot_all_routes(coordinates, cluster_routes_copy2, total_distance_copy2, f'Merged Routes After Step {merge_step} (Strategy 2)')
                merge_step += 1

final_route_copy2 = cluster_routes_copy2[0]

print(f'Final merged route (Strategy 2): {final_route_copy2}')
print(f'Number of points in final route (Strategy 2): {len(final_route_copy2)}')
plot_all_routes(coordinates, [final_route_copy2], total_distance_copy2, 'Final Merged Route (Strategy 2)')



FileNotFoundError: [Errno 2] No such file or directory: 'pr144.tsp'